In [2]:
import pandas as pd
import numpy as np
import json
import os
import gc
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="whitegrid")
import dask.dataframe as dd
from tqdm.auto import tqdm
from pandas.io.json import json_normalize

In [9]:
def process_subtable(tables):
    indexes = tables.index

    _list = []
    for idx, sub_table in tqdm(zip(indexes, tables), total=len(tables)):
        if (sub_table is None) or (sub_table is np.nan):
            pass
        else:
            df_temp = json_normalize(sub_table)
            df_temp['dadosBasicos.numero'] = idx
            _list.append(df_temp)
            
    return pd.concat(_list, axis=0, sort=False).set_index('dadosBasicos.numero')

# Processa amostras

## Dados Básicos

In [4]:
data_type = 'justica_estadual'
json_files = [
    file for file in glob(f'../data/{data_type}/*/*.json')
    if ('tjpr' in file) or ('tjrj' in file) or ('tjmg' in file)
]

base_path = f'../data/processed/samples/{data_type}/'
os.makedirs(base_path)

data_list = []
for file in tqdm(json_files, total=len(json_files)):
    with open(file) as f:
        data = json.load(f)
        filtered_data = [d for d in data if int(d['dadosBasicos']['classeProcessual']) == 1116]
        if len(filtered_data) == 0:
            continue
        df = json_normalize(filtered_data).set_index('dadosBasicos.numero')
        data_list.append(df)

In [5]:
df = pd.concat(data_list, sort=False, axis=0)
df.head()

,millisInsercao,siglaTribunal,movimento,grau,dadosBasicos.valorCausa,dadosBasicos.procEl,dadosBasicos.competencia,dadosBasicos.codigoLocalidade,dadosBasicos.assunto,dadosBasicos.dscSistema,dadosBasicos.tamanhoProcesso,dadosBasicos.dataAjuizamento,dadosBasicos.totalAssuntos,dadosBasicos.nivelSigilo,dadosBasicos.classeProcessual,dadosBasicos.orgaoJulgador.nomeOrgao,dadosBasicos.orgaoJulgador.codigoMunicipioIBGE,dadosBasicos.orgaoJulgador.codigoOrgao,dadosBasicos.orgaoJulgador.instancia
dadosBasicos.numero,,,,,,,,,,,,,,,,,,,
50006423320208130620,1596842405951,TJMG,"[{'identificadorMovimento': '1', 'tipoResponsa...",G1,2157.81,1.0,8.0,620,"[{'principal': True, 'assuntoLocal': None, 'co...",8,0.0,20200318060629,1.0,0,1116,"2ª Vara Cível, Criminal e da Infância e da Juv...",3162005,5843,ORIG
50006431820208130620,1596839821655,TJMG,"[{'identificadorMovimento': '1', 'tipoResponsa...",G1,751.18,1.0,8.0,620,"[{'principal': True, 'assuntoLocal': None, 'co...",8,0.0,20200318061146,1.0,0,1116,"2ª Vara Cível, Criminal e da Infância e da Juv...",3162005,5843,ORIG
50006440320208130620,1596840618466,TJMG,"[{'identificadorMovimento': '1', 'tipoResponsa...",G1,719.03,1.0,8.0,620,"[{'principal': True, 'assuntoLocal': None, 'co...",8,0.0,20200318061633,1.0,0,1116,"1ª Vara Cível, Criminal e de Execuções Penais ...",3162005,5836,ORIG
50006458520208130620,1596839978166,TJMG,"[{'identificadorMovimento': '1', 'tipoResponsa...",G1,1321.80,1.0,8.0,620,"[{'principal': True, 'assuntoLocal': None, 'co...",8,0.0,20200318062142,1.0,0,1116,"2ª Vara Cível, Criminal e da Infância e da Juv...",3162005,5843,ORIG
50006467020208130620,1596839983462,TJMG,"[{'identificadorMovimento': '1', 'tipoResponsa...",G1,1071.21,1.0,8.0,620,"[{'principal': True, 'assuntoLocal': None, 'co...",8,0.0,20200318063259,1.0,0,1116,"1ª Vara Cível, Criminal e de Execuções Penais ...",3162005,5836,ORIG


In [8]:
print(df.shape)
print(df[df['dadosBasicos.classeProcessual'] == 1116].shape)

(46998, 19)
(46998, 19)


In [6]:
df['siglaTribunal'].value_counts()

TJPR    17018
TJRJ    16847
TJMG    13133
Name: siglaTribunal, dtype: int64

## dadosBasicos.assunto

In [10]:
df_sub = process_subtable(df['dadosBasicos.assunto'])
print(df_sub.shape)
df_sub.head()


(77517, 9)


,principal,assuntoLocal,codigoNacional,assuntoLocal.codigoAssunto,assuntoLocal.codigoPaiNacional,assuntoLocal.descricao,codigoAssunto,codigoPaiNacional,descricao
dadosBasicos.numero,,,,,,,,,
50006423320208130620,True,None,5952.0,NaN,NaN,NaN,NaN,NaN,NaN
50006423320208130620,True,None,5952.0,NaN,NaN,NaN,NaN,NaN,NaN
50006431820208130620,True,None,5952.0,NaN,NaN,NaN,NaN,NaN,NaN
50006431820208130620,True,None,5952.0,NaN,NaN,NaN,NaN,NaN,NaN
50006440320208130620,True,None,5952.0,NaN,NaN,NaN,NaN,NaN,NaN


## movimento

In [11]:
df_moves = process_subtable(df['movimento'])
print(df_moves.shape)
df_moves.head()


(1720223, 18)


,identificadorMovimento,tipoResponsavelMovimento,complementoNacional,idDocumentoVinculado,movimentoLocal,nivelSigilo,dataHora,tipoDecisao,movimentoNacional.codigoNacional,orgaoJulgador.nomeOrgao,orgaoJulgador.codigoMunicipioIBGE,orgaoJulgador.codigoOrgao,orgaoJulgador.instancia,movimentoNacional,orgaoJulgador,movimentoLocal.movimentoLocalPai,movimentoLocal.codigoMovimento,movimentoLocal.codigoPaiNacional
dadosBasicos.numero,,,,,,,,,,,,,,,,,,
50006423320208130620,1,0,[{'descricaoComplemento': 'tipo_de_distribuica...,[],None,0,20200318060630,None,26.0,"2ª Vara Cível, Criminal e da Infância e da Juv...",3162005.0,5843.0,ORIG,NaN,NaN,NaN,NaN,NaN
50006423320208130620,2,0,"[{'descricaoComplemento': 'tipo_de_documento',...",[],None,0,20200319145437,None,60.0,"2ª Vara Cível, Criminal e da Infância e da Juv...",3162005.0,5843.0,ORIG,NaN,NaN,NaN,NaN,NaN
50006423320208130620,3,0,"[{'descricaoComplemento': 'tipo_de_conclusao',...",[],None,0,20200319145501,None,51.0,"2ª Vara Cível, Criminal e da Infância e da Juv...",3162005.0,5843.0,ORIG,NaN,NaN,NaN,NaN,NaN
50006423320208130620,4,1,[],[],None,0,20200324141733,None,11010.0,"2ª Vara Cível, Criminal e da Infância e da Juv...",3162005.0,5843.0,ORIG,NaN,NaN,NaN,NaN,NaN
50006423320208130620,5,0,"[{'descricaoComplemento': 'tipo_de_documento',...",[],None,0,20200610110943,None,60.0,"2ª Vara Cível, Criminal e da Infância e da Juv...",3162005.0,5843.0,ORIG,NaN,NaN,NaN,NaN,NaN


# Store data

In [13]:
df.drop(['movimento', 'dadosBasicos.assunto'], axis=1, inplace=True)
df.to_csv(f'{base_path}/dados_basicos.csv')
df_moves.to_csv(f'{base_path}/movimentos.csv')
df_sub.to_csv(f'{base_path}/assuntos.csv')